# Donut in Edward

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from timeit import default_timer as timer
import numpy as np
import pickle
import tensorflow as tf

import edward as ed
from edward.models import HalfNormal, Normal, Empirical

from utils import generate_datasets, SEED

In [ ]:
print(ed.__version__)

In [ ]:
ed.set_seed(42)

## 1. Model

In [ ]:
def Edward_model(n_samples, n_dim):
  R_e = HalfNormal(10.0)
  r_e = HalfNormal(10.0)
  C_e = Normal(0.0, 10.0, sample_shape = n_dim)
  v_e = Normal(0.0, 1.0, sample_shape = [n_samples, n_dim])
    
  y_e = Normal(loc = C_e + R_e*v_e/tf.norm(v_e, axis=1, keepdims=True), scale=r_e)
  return R_e, r_e, C_e, v_e, y_e

## 2. Inference

### Metropolis-Hastings

In [ ]:
def _Ed_MH(y, n_samples, n_dim, iters=8000, burn=4000):
  """
  Runs Edward's Metropolis-Hastings algorithm for one seed
  """
  R_e, r_e, C_e, v_e, y_e = Edward_model(n_samples, n_dim)
    
  qR = Empirical(params=tf.Variable(tf.ones(iters) * .5))
  qr = Empirical(params=tf.Variable(tf.ones(iters) * .5))
  qC = Empirical(params=tf.Variable(tf.ones([iters,n_dim]) * .5))

  R_proposal = Normal(loc=R_e, scale=0.1)
  r_proposal = Normal(loc=r_e, scale=0.1)
  C_proposal = Normal(loc=C_e, scale=0.1)

  inference = ed.MetropolisHastings({R_e: qR, r_e: qr, C_e: qC}, 
                                    {R_e: R_proposal, r_e: r_proposal, C_e: C_proposal}, data={y_e: y})
    
  start = timer()
  inference.run()
  end = timer()
    
  return qR.params.eval()[burn:], qr.params.eval()[burn:], qC.params.eval()[burn:], end-start

In [ ]:
# Metropolis-Hastings
def Ed_MH(n_samples, n_dim, iters=8000, burn=4000, seeds=SEED):
  """
  Runs Edward's Metropolis-Hastings algorithm for each seed
  """
  Y, C, R, r = generate_datasets(n_samples, n_dim, seeds)
  results = {}
  for seed,y in zip(seeds, Y):
    qR, qr, qC, time = _Ed_MH(y, n_samples, n_dim, iters, burn)
    results = {'time': time, 'R': qR, 'r': qr, 'C': qC, 'iters': iters, 'burn': burn}
    with open('results/edward/mh_{}d_{}.pkl'.format(n_dim, seed), 'wb') as f:
      pickle.dump(results, f)
  print('Done')

In [ ]:
# Small dataset
n_samples = 1000
n_dim = 2
Ed_MH(n_samples, n_dim)

In [ ]:
# Big dataset
n_samples = 5000
n_dim = 5
Ed_MH(n_samples, n_dim)

### Klqp

In [ ]:
def _Ed_VI(y, n_samples, n_dim, iters):
  """
  Runs Edward's KLqp algorithm, for one seed 
  """
  R_e, r_e, C_e, v_e, y_e = Edward_model(n_samples, n_dim)
    
  with tf.variable_scope('qR', reuse=tf.AUTO_REUSE):
    qR = HalfNormal(tf.nn.softplus(tf.get_variable('scale', [])))
    
  with tf.variable_scope('qr', reuse=tf.AUTO_REUSE):
    qr = HalfNormal(tf.nn.softplus(tf.get_variable('scale', [])))

  with tf.variable_scope('qC', reuse=tf.AUTO_REUSE):
    qC = Normal(tf.get_variable('loc', [n_dim]),
                      tf.nn.softplus(tf.get_variable('scale', [n_dim])))

  inference = ed.KLqp({R_e: qR, r_e: qr, C_e: qC}, data={y_e: y})
    
  start = timer()
  inference.run(n_samples=10, n_iter=iters)
  end = timer()
    
  return qR, qr, qC, end-start

In [ ]:
def Ed_VI(n_samples, n_dim, seeds=SEED):
  """
  Runs Edward's ADVI algorithm, for each seed 
  """
  Y, C, R, r = generate_datasets(n_samples, n_dim, seeds)
  for y,seed in zip(Y, seeds):
    print(seed)
    iters = np.linspace(1000, 50000, 5).astype(int)
    for n in iters:
      qR, qr, qC, time = _Ed_VI(y, n_samples, n_dim, n)
      print(time)
      
      results = {'time': time, 'iters': n, 'R': qR.sample(1000).eval()[:,None], 'r': qr.sample(1000).eval()[:,None], 'C': qC.sample(1000).eval()} 
      with open('results/edward/vi_{}d_{}.pkl'.format(n_dim, seed), 'ab') as f:
        pickle.dump(results, f)
  print('Done')

In [ ]:
# Small dataset
n_samples = 1000
n_dim = 2
Ed_VI(n_samples, n_dim)

In [ ]:
# Big dataset
n_samples = 5000
n_dim = 5
Ed_VI(n_samples, n_dim)